In [ ]:
mkdir fastq  key  output  referenceGenome
#Create DB
#/bin/bash
ml tassel
cwd=$(pwd)
barcodes=${cwd}/barcode-for-tassel.tsv
db=${cwd}/GBSV2.db
run_pipeline.pl -Xms100G -Xmx180G -fork1 -GBSSeqToTagDBPlugin \
   -e HindIII \
   -i ${cwd} \
   -db ${db} \
   -k ${barcodes}  \
   -kmerLength 85 \
   -minKmerL 30 \
   -mnQS 33 \
   -mxKmerNum 1000000000 \
   -endPlugin \
   -runfork1
#Tagexport
#/bin/bash
ml tassel
cwd=$(pwd)
barcodes=${cwd}/barcode-for-tassel.tsv
db=${cwd}/GBSV2.db
run_pipeline.pl -Xms100G -Xmx180G -fork1 -TagExportToFastqPlugin \
 -db ${db} \
 -o ${cwd}/tagsForAlign.fa.gz \
 -c 5 \
 -endPlugin \
 -runfork1
#Alignment
ml bowtie2
cwd=$(pwd)
barcodes=${cwd}/barcode-for-tassel.tsv
db=${cwd}/GBSV2.db
btindex="/ref/scaffolds.fa"
bowtie2 -p 15 --very-sensitive -x ${btindex} -U ${cwd}/tagsForAlign.fa.gz -S tagsForAlignFullvs.sam
SAM="tagsForAlignFullvs.sam"
samtools view --threads 16 -b -o ${SAM%.*}.bam ${SAM}
samtools sort -o ${SAM%.*}_sorted.bam -T ${SAM%.*}_temp --threads 36 ${SAM%.*}.bam
#SamtoGBS
#/bin/bash
ml tassel
cwd=$(pwd)
barcodes=${cwd}/barcode-for-tassel.tsv
db=${cwd}/GBSV2.db
run_pipeline.pl -Xms100G -Xmx180G -fork1 -SAMToGBSdbPlugin \
  -i ${cwd}/tagsForAlignFullvs.sam \
  -db ${db} \
  -aProp 0.0 \
  -aLen 0 \
  -endPlugin \
  -runfork1
#Discovery SNP
#/bin/bash
ml tassel
cwd=$(pwd)
barcodes=${cwd}/barcode-for-tassel.tsv
db=${cwd}/GBSV2.db
run_pipeline.pl -Xms100G -Xmx180G -fork1 -DiscoverySNPCallerPluginV2 \
  -db ${db} \
  -sC 0 \
  -eC 0 \
  -mnLCov 0.1 \
  -mnMAF 0.01 \
  -deleteOldData true \
  -endPlugin \
  -runfork1

    
#Production SNP caller

#!/bin/bash
ml tassel
cwd=$(pwd)
barcodes=${cwd}/barcode-for-tassel.tsv
db=${cwd}/GBSV2.db
run_pipeline.pl -Xms100G -Xmx180G -fork1 -ProductionSNPCallerPluginV2 \
  -db ${db} \
  -e HindIII \
  -i ${cwd} \
  -k ${barcodes} \
  -kmerLength 33 \
  -o ${cwd}/tassel-gbs.vcf \
  -endPlugin \
  -runfork1

#Or h5 format
run_pipeline.pl -Xms100G -Xmx180G -fork1 -ProductionSNPCallerPluginV2 \
  -db ${db} \
  -e ApeKI \
  -i ${cwd} \
  -k ${barcodes} \
  -kmerLength 33 \
  -o ${cwd}/tassel-gbs.h5 \
  -endPlugin \
  -runfork1
